In [1]:
import parse_tfl_json

In [10]:
import pandas as pd 
from mylibrary.connections import cursor, conn, engine, Automapped_Base, session

sql = """
select * from tt_h.stations_journeytime_cw where tfl_message = 'ok' limit 30
"""
df = pd.read_sql(sql, conn)

In [11]:
import json 

for r in df.iterrows():
    index = r[0]
    row = r[1]

    j1 = row["tfl_response"]
    j2 = json.loads(j1)
    
    journeys = j2["journeys"]
    journey = sorted(journeys, key = lambda x:  x["duration"])[0]
    
    legs = journey["legs"]
    
    legs = parse_tfl_json.remove_non_national_rail(legs)
    
    summary = parse_tfl_json.summarise_legs(legs)
    df.loc[index, "natrail_journey_summary"] = summary
    
    df.loc[index, "natrail_train_changes"] = parse_tfl_json.num_changes(legs)
    df.loc[index, "final_arrival"] = parse_tfl_json.final_arrival(legs)
    
    travel_time_minutes = parse_tfl_json.get_total_travel_time(legs) 
    df.loc[index, "natrail_journey_minutes"] = travel_time_minutes
    
    lat_lng = parse_tfl_json.lat_lng_from_legs(legs)
    df.loc[index, "depart_lat"] = lat_lng["depart"]["lat"]
    df.loc[index, "depart_lng"] = lat_lng["depart"]["lng"]
    df.loc[index, "arrive_lat"] = lat_lng["arrive"]["lat"]
    df.loc[index, "arrive_lng"] = lat_lng["arrive"]["lng"]
    
    
    
cols = [c for c in df.columns if c not in ["id", "tfl_request", "tfl_response", "querydict", "journey_time", "london_mainline_station_ics", "london_mainline_station_text"]]
df[cols]

,nlc,icscode,station_name,tfl_message,natrail_journey_summary,natrail_train_changes,final_arrival,natrail_journey_minutes,depart_lat,depart_lng,arrive_lat,arrive_lng
0,6846,1000691,White Notley,ok,White Notley Rail Station → Stratford (London)...,0.0,Stratford (London) Rail Station,44.0,51.838910,0.595863,51.542234,-0.002819
1,5686,90008069,Worplesdon,ok,Worplesdon Rail Station → Waterloo Station,0.0,Waterloo Station,33.0,51.289016,-0.582578,51.502346,-0.114373
2,6551,90007791,Whitwell (Derbys),ok,Whitwell (Derbys) Rail Station → Worksop Rail ...,3.0,London King's Cross Rail Station,181.0,53.279959,-1.200207,51.531579,-0.123243
3,5827,90008172,Yeoford,ok,Yeoford Rail Station → Exeter St Davids Rail S...,1.0,Paddington Station,173.0,50.776893,-3.726054,51.516993,-0.177388
4,5686,90008069,Worplesdon,ok,Worplesdon Rail Station → Waterloo Station,0.0,Waterloo Station,33.0,51.289016,-0.582578,51.502346,-0.114373
5,9665,90007761,Whinhill,ok,Whinhill Rail Station → Glasgow Central Rail S...,2.0,Euston Station,351.0,55.938370,-4.746694,51.528952,-0.135000
6,7227,90007629,Wickham Market,ok,Wickham Market Rail Station → Ipswich Rail Sta...,1.0,Stratford (London) Rail Station,98.0,52.151099,1.398680,51.542234,-0.002819
7,8263,90008178,York,ok,York Rail Station → London King's Cross Rail S...,0.0,London King's Cross Rail Station,114.0,53.957964,-1.093182,51.531579,-0.123243
8,6551,90007791,Whitwell (Derbys),ok,Whitwell (Derbys) Rail Station → Worksop Rail ...,3.0,London King's Cross Rail Station,181.0,53.279959,-1.200207,51.531579,-0.123243
9,6551,90007791,Whitwell (Derbys),ok,Whitwell (Derbys) Rail Station → Worksop Rail ...,3.0,London King's Cross Rail Station,181.0,53.279959,-1.200207,51.531579,-0.123243


In [8]:
# Next - need a bikedistance function that takes a lat lng and looks up 



"Whitwell (Derbys) Rail Station → Worksop Rail Station → Retford Low Level Rail Station → Retford Rail Station → London King's Cross Rail Station"